# Plase Download loan.csv from below given URL.
https://www.kaggle.com/sudalairajkumar/getting-started-with-python-datatable/notebook#Reading-Data

In [3]:
import time
import numpy as np
import pandas as pd
import datatable as dt


ModuleNotFoundError: No module named 'datatable'

Reading Data
Let us first start with reading the input data from csv file. loan.csv file has about 2.26 million rows and 145 columns.

fread is the fucntion to read the files in datatable module. From the documentation,fread provides fast and convenient parsing of text (csv) files.

Automatically detects separators, headers, column types, quoting rules, etc.
Reads from file, URL, shell, raw text, archives, glob
Provides multi-threaded file reading for maximum speed
Includes a progress indicator when reading large files
Reads both RFC4180-compliant and non-compliant files

In [2]:
## Data Table Reading
start = time.time()
dt_df = dt.fread("loan.csv")
end = time.time()
print(end - start)

NameError: name 'dt' is not defined

This took about ~6 seconds to read the data and create a Frame object. The fundamental unit of analysis in datatable is a data Frame. It is the same notion as a pandas DataFrame or SQL table: data arranged in a two-dimensional array with rows and columns.

Now let us read the same file in pandas as see the time difference.

In [3]:
start = time.time()
pd_df= pd.read_csv("loan.csv")
end = time.time()
print(end - start)

/Users/rakeshkumar/miniconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (19,47,55,112,123,124,125,128,129,130,133,139,140,141) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


47.102200984954834


Pandas takes more than ~45 seconds to read the same file and create a dataframe from it. I think this will be a good value add when we deal with large datasets.

We also have options in datatable to convert the frame to pandas dataframe object. Let us do that.

In [4]:
start = time.time()
dt_df.to_pandas()
end = time.time()
print(end - start)

28.90583872795105


In [5]:
type(pd_df)

pandas.core.frame.DataFrame

In [6]:
type(dt_df)

datatable.Frame

Looks like this takes ~25 seconds. So reading a file as datable frame and converting that to pandas dataframe is faster than reading through pandas directly ;)

# Frame Properties
Similar to pandas, we can get the

top few rows by head command,
size of the frame by shape command and
column names by names command

In [7]:
print(dt_df.head(5))

<Frame [5 rows x 145 cols]>


In [8]:
print(dt_df.shape)

(2260668, 145)


In [9]:
print(dt_df.names)

('id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose', 'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'policy_code', 'application_type', 'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 'open_act_il', 'open_il_12m', 'open_i

# Summary Statistics
We can also get the summary statistics like sum, mean, max, min, sd etc.

Let us get the mean of the columns using both datatable and pandas

In [10]:
print(dt_df.mean)

<bound method Frame.mean of <Frame [2,260,668 rows x 145 cols]>>


In [11]:
print(dt_df.mean())

<Frame [1 row x 145 cols]>


When I ran the below one, I got memory error and so commenting it out.

In [12]:
# print(pd_df.mean())

# Sorting the frame
Now let us sort the dataframe by loan_amnt column.

In [13]:
start = time.time()
dt_df.sort("loan_amnt")
end = time.time()
print(end - start)

0.023934364318847656


In [14]:
start = time.time()
pd_df.sort_values(by="loan_amnt")
end = time.time()
print(end - start)

13.397078037261963


Wow! This is super fast.

# GroupBy Operations
Let us get the mean loan amount for each grades.

In datatable, almost all operations with a Frame can be expressed as

DT[ i, j, ...]

where

i is the row selector

j is the column selector and

... represents additional modifiers might be added

In [15]:
start = time.time()
for i in range(100):
    dt_df[:, dt.sum(dt.f.loan_amnt), dt.by(dt.f.grade)]
end = time.time()
print(end - start)

1.3400766849517822


In this case i is all the rows, j is loan_amnt and ... is grouping by grade

We have also used dt.f in the code - what does it mean? It is a "frame proxy", as it becomes a simple way to refer to the Frame that we currently operate on. In this example, dt.f represents the frame dt_df

In [16]:
start = time.time()
for i in range(100):
    pd_df.groupby("grade")["loan_amnt"].sum()
end = time.time()
print(end - start)

10.027663946151733


# Filtering rows
Filtering also follows the same syntax as that of groupby

Let us filter those rows which has loan_amnt greater than the mean loan_amnt.

In [17]:
start = time.time()
for i in range(100):
    dt_df[dt.f.loan_amnt>dt.mean(dt.f.loan_amnt), "loan_amnt"]
end = time.time()
print(end - start)

2.6437559127807617


In [18]:
print(pd_df["loan_amnt"])

0           2500
1          30000
2           5000
3           4000
4          30000
5           5550
6           2000
7           6000
8           5000
9           6000
10          5500
11         28000
12         11200
13          6500
14         22000
15          3500
16          7000
17         25000
18         16000
19         13000
20         10000
21         13000
22          9600
23          3500
24         16000
25         15000
26         13000
27         23000
28          8000
29         32075
           ...  
2260638    30000
2260639    21000
2260640    20700
2260641    15000
2260642    32400
2260643    10000
2260644    35000
2260645    20000
2260646    16000
2260647    10500
2260648    32000
2260649    35000
2260650    10000
2260651    35000
2260652    25000
2260653    15000
2260654    30000
2260655    18000
2260656    17950
2260657    19000
2260658    15000
2260659     3500
2260660    10000
2260661    21000
2260662    30000
2260663    12000
2260664    12000
2260665    100

In [19]:
start = time.time()
for i in range(100):
    pd_df["loan_amnt"][pd_df["loan_amnt"] > pd_df["loan_amnt"].mean()]
end = time.time()
print(end - start)

4.0381388664245605


In [20]:
#pd_df["loan_amnt"]

In [21]:
print(pd_df["loan_amnt"].shape)

(2260668,)


# Saving the Frame
Now let us look at the ways to save the frame so as to load back when needed. datatable has to_csv function similar to pandas.

In [22]:
gdf = dt_df[:, dt.sum(dt.f.loan_amnt), dt.by(dt.f.grade)]
gdf.to_csv("temp_dataTable.csv")

In [23]:
new_pd_data = pd_df.groupby("grade")["loan_amnt"].sum()
new_pd_data.to_csv("temp_pandas.csv")

In [28]:
print("Saving Frame For DataTable : ",gdf)
print(gdf.names)


Saving Frame For DataTable :  <Frame [7 rows x 2 cols]>
('grade', 'C0')
<Frame [5 rows x 2 cols]>


In [32]:
print("Saving Frame For Pandas : ",new_pd_data)
print("Shape : ",new_pd_data.shape)

Saving Frame For Pandas :  grade
A    6323641900
B    9404817775
C    9775551175
D    5097344375
E    2367318100
F     799410225
G     248032375
Name: loan_amnt, dtype: int64
Shape :  (7,)
